In [ ]:
%run ndvi.py
%run mean_sst.py

In [ ]:
import xarray as xr
import requests as rq
import json
import hvplot.xarray

## Server

#### Verwalten von Jobs

Die Jobs können jetzt gepatch werden, falls der Nutzer einem Job nochmal Änderungen hinzufügen möchte. 

Ein Beispiel:

In [ ]:
lamejob = {
  "title": "Lame Title",
  "description": "Example Description",
  "process": {
    "process_graph": {
      "loadcollection1": {
        "process_id": "load_collection",
        "arguments": {
          "timeframe" : ["01-06-2020 00:00:00","10-06-2020 00:00:00","%d-%m-%Y %H:%M:%S"],
          "DataType": "Sentinel2",
          "cloudcoverage":[0,30],
          "Login":['username', 'pw']
        }
        },
        "ndvi": {
        "process_id": "ndvi",
        "arguments": {
          "data":{
              "from_node": "loadcollection1"
          }
        }
        },
        "save":{
            "process_id": "save_result",
            "arguments":{
                "SaveData":{
                    "from_node":"ndvi"
                },
                "Format": "netcdf"
            }
        }
      }
      }
    }

Posten des Jobs:

In [ ]:
#Den Testdaten an /jobs Posten
rq.post("http://localhost:8080/api/v1/jobs", json=lamejob, headers={"Content-Type": "application/json"})

Abrufen der Job Id:

In [ ]:
j = rq.get("http://localhost:8080/api/v1/jobs")
rjson = j.json()
# Die Id des neusten Job abspeichern für spätere Verwendung
job_id = rjson['jobs'][-1]['id']
rjson

Verbesster Job:

In [ ]:
cooljob = {
  "title": "Cool Title",
  "description": "Example Description",
  "process": {
    "process_graph": {
      "loadcollection1": {
        "process_id": "load_collection",
        "arguments": {
          "timeframe" : ["01-06-2020 00:00:00","10-06-2020 00:00:00","%d-%m-%Y %H:%M:%S"],
          "DataType": "Sentinel2",
          "cloudcoverage":[0,30],
          "Login":['username', 'pw']
        }
        },
        "ndvi": {
        "process_id": "ndvi",
        "arguments": {
          "data":{
              "from_node": "loadcollection1"
          }
        }
        },
        "save":{
            "process_id": "save_result",
            "arguments":{
                "SaveData":{
                    "from_node":"ndvi"
                },
                "Format": "netcdf"
            }
        }
      }
      }
    }

Die Patch Request:

In [ ]:
rq.patch("http://localhost:8080/api/v1/jobs/" + job_id  , json=cooljob, headers={"Content-Type": "application/json"})

Der veränderte Job in der Jobübersicht

In [ ]:
j = rq.get("http://localhost:8080/api/v1/jobs")
rjson = j.json()
rjson

## Datenverwaltung

Die Anwendung lädt jetzt schon bei `docker-compose up` die gewünschten Datenmengen herunter. Dies lässt sich über die Enviroment Variable im docker-compose.yml einstellen. Hier einmal die Einstellungen für diese Demo:

```yml
environment:
      - DOCKER=True
      - Username=bugmenot
      - Password=bugmenot
      - load_sst=False
      - load_sentinel=False
      - SST_Start=2012
      - SST_End=2012
      - Sentinel_Start=2020-06-01T00:00:00Z
      - Sentinel_End=2020-06-10T00:00:00Z
```

Auch ein Mount von lokalen Dateien ist möglich. 

## NDVI Bug

Der NDVI Bug von letzter Woche ist behoben. Wir haben vorher schonmal einen Job durchlaufen lassen, um das zu zeigen. Die Umrechnung mit -1 ist nicht mehr nötig und der Server liefert so das richtige Ergebnis.

In [ ]:
ndvi = xr.open_dataset('Euer Downloadpfad')
ndvi.hvplot(cmap="BrBG",frame_height=549, frame_width=549)

Damit dieses Beispiel rekreiieren werden kann folgt diesen Schritten:

1. Führt `docker-compose up` in den Verzeichnis Demo_VI aus.
2. Legt folgendes Job JSON an und setzt eure SCIHUB Kontoinformationen:
```JSON
{
  "title": "Example Title",
  "description": "Example Description",
  "process": {
    "process_graph": {
      "loadcollection1": {
        "process_id": "load_collection",
        "arguments": {
          "timeframe" : ["01-06-2020 00:00:00","01-06-2020 00:00:00","%d-%m-%Y %H:%M:%S"],
          "DataType": "Sentinel2",
          "cloudcoverage":[0,30],
          "Login":[<SCIHUB USERNAME>, <SCIHUB PASSWORT>]
        }
        },
        "ndvi": {
        "process_id": "ndvi",
        "arguments": {
          "data":{
              "from_node": "loadcollection1"
          }
        }
        },
        "save":{
            "process_id": "save_result",
            "arguments":{
                "SaveData":{
                    "from_node":"ndvi"
                },
                "Format": "netcdf"
            }
        }
      }
      }
    }
```
3. Folgt den Schritten für das ausführen eines Jobs in Demo IV und V.

## SST 

#### Größere Berechnung

Um die Vorteile der SST des Einsatzes von Dask zu zeigen, wird im folgenden eine Berechnung über einen Zeitraum von Vier Jahren gemacht(2016-2019)

In [ ]:
big_cube = xr.open_dataset('../demodata/SSTdatacube.nc', chunks='auto')
big_cube.sst

In [ ]:
%%time
sst_16_19 = wrapper_mean_sst(big_cube, ["2016-01-01", "2019-12-31"])

In [ ]:
sst_16_19.hvplot(cmap="coolwarm")

## Tests

#### End-To-End Test

Der [End-To-End Testflow](https://github.com/GeoSoftII2020-21/TestRepo/runs/1797232156?check_suite_focus=true) läuft jetzt durch. Dabei werden zwei Jobs (SST und NDVI) angelegt.  
Dann werden die die Endpoints der API überprüft und die Jobs gestartet. Der Ablauf und die Ergebnisse der Jobs werden dann auch geprüft. Die Anzahl der Werte wird am Ende überprüft.